In [1]:
!wget -P data/dealership https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2023-06-13 23:54:03--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 

169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4,1K) [application/zip]
Saving to: ‘data/dealership/datasource.zip’

datasource.zip      100%[===================>]   4,15K  --.-KB/s    in 0s      

2023-06-13 23:54:04 (899 MB/s) - ‘data/dealership/datasource.zip’ saved [4249/4249]



In [2]:
!unzip data/dealership/datasource.zip -d data/dealership/source

Archive:  data/dealership/datasource.zip
  inflating: data/dealership/source/used_car_prices1.csv  
  inflating: data/dealership/source/used_car_prices2.csv  
  inflating: data/dealership/source/used_car_prices3.csv  
  inflating: data/dealership/source/used_car_prices1.json  
  inflating: data/dealership/source/used_car_prices2.json  
  inflating: data/dealership/source/used_car_prices3.json  
  inflating: data/dealership/source/used_car_prices1.xml  
  inflating: data/dealership/source/used_car_prices2.xml  
  inflating: data/dealership/source/used_car_prices3.xml  


In [3]:
import pandas as pd
import glob
from datetime import datetime

In [4]:
path_folder = "data/dealership"
source_folder = path_folder + "/source"
log_file = path_folder + "/log.txt"
target_file = path_folder + "/dealership.csv" 

### Extract


In [5]:
def extract_from_csv(file_to_process):
    df = pd.read_csv(file_to_process)
    return df

def extract_from_json(file_to_process):
    df = pd.read_json(file_to_process, lines=True)
    return df

def extract_from_xml(file_to_process):
    df = pd.read_xml(file_to_process)
    return df

In [6]:
def extract(folder):
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])

    for csvfile in glob.glob(folder + "/*.csv"):
        extracted_data = pd.concat([extracted_data, extract_from_csv(csvfile)], ignore_index=True)

    for jsonfile in glob.glob(folder + "/*.json"):
        extracted_data = pd.concat([extracted_data, extract_from_json(jsonfile)], ignore_index=True)
    
    for xmlfile in glob.glob(folder + "/*.xml"):
        extracted_data = pd.concat([extracted_data, extract_from_xml(xmlfile)], ignore_index=True)
    
    return extracted_data

### Transform

In [7]:
def transform(data):
    data['price'] = round(data.price, 2)
    return data

### Loading

In [8]:
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

### Logging

In [9]:
def log(message):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, 'a') as log:
        log.write(now + " " + message + "\n")

#### Running

In [10]:
log("ETL Job Started")

log("Extract phase Started")
extracted_data = extract(source_folder)
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(target_file,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")